In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import pandas as pd; pd.set_option('display.max_rows', None)
import glob
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import sys
sys.path.append('scripts/')
from Acompanhamento import *
METODO = 'over_v0'
TARGET = 'Back_Over'
ODDS = 'Odds_Over'
SPREAD = 1
STAKE = 1


# Dados de backtest
wr = 56
roi = 15.15
drawdown = 0
med_gain = 1.07
med_loss = -1.00
ev = 0.10
df_hist = pd.read_excel('data/metodos_hist/v0_hist.xlsx')
df_hist['Date'] = pd.to_datetime(df_hist['Date']).dt.date
df_hist.loc[(df_hist['Resultado'] == 'green'), TARGET] = 1
df_hist.loc[(df_hist['Resultado'] == 'red'), TARGET] = 0
df_all = pd.DataFrame()
dias = glob.glob('apostas_do_dia/*')

for dia in dias:
    dia = dia.split('\\')[1]
    dia = dia.split('.')[0]
    caminho = glob.glob(f'apostas_do_dia/{dia}/*{METODO}*')
    if len(caminho) == 0:
        continue
    caminho = caminho[0]
    df = pd.read_csv(caminho)
    df_all = pd.concat([df_all, df])

temp = pd.read_csv('data/Entire_Season.csv')
df_all = pd.merge(df_all, temp[['Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], on=['Date', 'Home', 'Away'], how='left')
df_all = df_all.dropna(subset=['Home_Pts', 'Away_Pts'])
df_all['Home_Pts'] = df_all['Home_Pts'].astype(int)
df_all['Away_Pts'] = df_all['Away_Pts'].astype(int)

df_all.loc[((df_all['Home_Pts'] + df_all['Away_Pts']) > df_all['Over_Line']), TARGET] = 1
df_all.loc[((df_all['Home_Pts'] + df_all['Away_Pts']) < df_all['Over_Line']), TARGET] = 0
df_all = pd.concat([df_hist, df_all])
df_all.reset_index(drop=True, inplace=True)

df_all.loc[(df_all[TARGET] == 0), 'Profit'] = - STAKE
df_all.loc[(df_all[TARGET] == 1), 'Profit'] = ((df_all[ODDS] * SPREAD) - 1) * STAKE
df_all['Profit'] = df_all['Profit'].round(2)

print('BACKTEST -----------------------')
print(f'WR: {wr:.2f}% | ROI: {roi:.2f}% | Drawdown: {drawdown:.2f}%')
print(f'MedGain: {med_gain:.2f} | MedLoss: {med_loss:.2f} | EV: {ev:.2f}')

print('\nREALIDADE -----------------------')
df_odds, df_by_day = get_results(df_all, TARGET, ODDS)

print('\nDias --------------------------------------')
display(df_by_day)
print('\nJogos -------------------------------------')
display(df_odds)

BACKTEST -----------------------
WR: 56.00% | ROI: 15.15% | Drawdown: 0.00%
MedGain: 1.07 | MedLoss: -1.00 | EV: 0.10

REALIDADE -----------------------
WR: 56.49% | ROI: 6.95% | Drawdown: -9.10%
MedGain: 0.89 | MedLoss: -1.00 | EV: 0.07

Profit: 33.23 | Entradas: 478
Dias: 46 (46 totais)



Dias --------------------------------------


Date  Total_Profit  Odds_Over  Acumulado
0   2023-10-20          0.64          7       0.64
1   2023-10-21         -0.81         18      -0.17
2   2023-10-22          3.84         23       3.67
3   2023-10-23          0.78          3       4.45
4   2023-10-24          1.68          6       6.13
5   2023-10-25          1.14         18       7.27
6   2023-10-26          0.55          9       7.82
7   2023-10-27         -0.75         14       7.07
8   2023-10-28         -4.03         25       3.04
9   2023-10-29          0.00         23       3.04
10  2023-10-30          4.37          9       7.41
11  2023-10-31          2.59          7      10.00
12  2023-11-01          2.28         13      12.28
13  2023-11-02          1.37         12      13.65
14  2023-11-03          1.32         14      14.97
15  2023-11-04          4.69         24      19.66
16  2023-11-05         -3.31         26      16.35
17  2023-11-06          1.42         48      17.77
18  2023-11-07          1.82          2      19.59
19  2023-11-08          5.49         13      25.08
20  2023-11-09          0.52          5      25.60
21  2023-11-10          0.83          1      26.43
22  2023-11-11         -1.14          3      25.29
23  2023-11-12          0.25          9      25.54
24  2023-11-13         -1.27          5      24.27
25  2023-11-14          1.74          2      26.01
26  2023-11-15          2.46          7      28.47
27  2023-11-16         -0.27          4      28.20
28  2023-11-17          4.27          9      32.47
29  2023-11-18          0.21          9      32.68
30  2023-11-19         -0.18          4      32.50
31  2023-11-20          1.42          4      33.92
32  2023-11-21         -0.14          2      33.78
33  2023-11-22         -4.36         10      29.42
34  2023-11-23          0.58          5      30.00
35  2023-11-24         -0.01         17      29.99
36  2023-11-25         -1.00          1      28.99
37  2023-11-26          0.67          7      29.66
38  2023-11-27         -0.31          4      29.35
39  2023-11-28         -1.43          7      27.92
40  2023-11-29         -0.46          6      27.46
41  2023-11-30          2.40          5      29.86
42  2023-12-01          1.53          4      31.39
43  2023-12-02          2.20         26      33.59
44  2023-12-03          0.64          7      34.23
45  2023-12-04         -1.00          1      33.23


Jogos -------------------------------------


Date                    Home                    Away  Home_Pts  \
0    2023-10-20                     NaN                     NaN       NaN   
1    2023-10-20                     NaN                     NaN       NaN   
2    2023-10-20                     NaN                     NaN       NaN   
3    2023-10-20                     NaN                     NaN       NaN   
4    2023-10-20                     NaN                     NaN       NaN   
5    2023-10-20                     NaN                     NaN       NaN   
6    2023-10-20                     NaN                     NaN       NaN   
7    2023-10-21                     NaN                     NaN       NaN   
8    2023-10-21                     NaN                     NaN       NaN   
9    2023-10-21                     NaN                     NaN       NaN   
10   2023-10-21                     NaN                     NaN       NaN   
11   2023-10-21                     NaN                     NaN       NaN   
12   2023-10-21                     NaN                     NaN       NaN   
13   2023-10-21                     NaN                     NaN       NaN   
14   2023-10-21                     NaN                     NaN       NaN   
15   2023-10-21                     NaN                     NaN       NaN   
16   2023-10-21                     NaN                     NaN       NaN   
17   2023-10-21                     NaN                     NaN       NaN   
18   2023-10-21                     NaN                     NaN       NaN   
19   2023-10-21                     NaN                     NaN       NaN   
20   2023-10-21                     NaN                     NaN       NaN   
21   2023-10-21                     NaN                     NaN       NaN   
22   2023-10-21                     NaN                     NaN       NaN   
23   2023-10-21                     NaN                     NaN       NaN   
24   2023-10-21                     NaN                     NaN       NaN   
25   2023-10-22                     NaN                     NaN       NaN   
26   2023-10-22                     NaN                     NaN       NaN   
27   2023-10-22                     NaN                     NaN       NaN   
28   2023-10-22                     NaN                     NaN       NaN   
29   2023-10-22                     NaN                     NaN       NaN   
30   2023-10-22                     NaN                     NaN       NaN   
31   2023-10-22                     NaN                     NaN       NaN   
32   2023-10-22                     NaN                     NaN       NaN   
33   2023-10-22                     NaN                     NaN       NaN   
34   2023-10-22                     NaN                     NaN       NaN   
35   2023-10-22                     NaN                     NaN       NaN   
36   2023-10-22                     NaN                     NaN       NaN   
37   2023-10-22                     NaN                     NaN       NaN   
38   2023-10-22                     NaN                     NaN       NaN   
39   2023-10-22                     NaN                     NaN       NaN   
40   2023-10-22                     NaN                     NaN       NaN   
41   2023-10-22                     NaN                     NaN       NaN   
42   2023-10-22                     NaN                     NaN       NaN   
43   2023-10-22                     NaN                     NaN       NaN   
44   2023-10-22                     NaN                     NaN       NaN   
45   2023-10-22                     NaN                     NaN       NaN   
46   2023-10-22                     NaN                     NaN       NaN   
47   2023-10-22                     NaN                     NaN       NaN   
48   2023-10-23                     NaN                     NaN       NaN   
49   2023-10-23                     NaN                     NaN       NaN   
50   2023-10-23                     NaN                     NaN       NaN   
51   20